In [15]:
from pathlib import Path
import shutil
import numpy as np

In [16]:
RUN_DIR  = Path("/project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/runs")
MOLD_DIR = Path("/project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/mold")
ANG_DIR  = Path("/project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/angles/runs")
NUM_SUB = 4
NWINDOWS_DIST = 12
NWINDOWS_ANG  = 18

# Preparations

In [17]:
def write_plumed(id, dist, ang, k_d, k_a, circ_id, plumed_f):
    plumed_cnt = [ "WHOLEMOLECULES ENTITY0=99-151 ENTITY1=1-98\n",
                   "# DEFINITIONS\n",
                   "MOLcom: COM ATOMS=99-151\n",
                   "PCCcom: COM ATOMS=1-98\n",
                   "# COLVARS\n",
                   "dcom: DISTANCE ATOMS=MOLcom,PCCcom\n",
                   "ang: ANGLE ATOMS=5,8,101,129\n",
                   "v1: DISTANCE ATOMS=5,8 SCALED_COMPONENTS NOPBC\n",
                   "v2: DISTANCE ATOMS=101,129 SCALED_COMPONENTS NOPBC\n",
                   "vr: DISTANCE ATOMS=PCCcom,2 SCALED_COMPONENTS NOPBC\n",
                   "crossx: CUSTOM ...\n",
                   "\tARG=v1.b,v1.c,v2.b,v2.c\n",
                   "\tVAR=y1,z1,y2,z2\n",
                   "\tFUNC=y1*z2-y2*z1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "crossy: CUSTOM ...\n",
                   "\tARG=v1.a,v1.c,v2.a,v2.c\n",
                   "\tVAR=x1,z1,x2,z2\n",
                   "\tFUNC=x2*z1-x1*z2\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "crossz: CUSTOM ...\n",
                   "\tARG=v1.a,v1.b,v2.a,v2.b\n",
                   "\tVAR=x1,y1,x2,y2\n",
                   "\tFUNC=x1*y2-x2*y1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "crosss: CUSTOM ...\n",
                   "\tARG=crossx,crossy,crossz\n",
                   "\tVAR=x,y,z\n",
                   "\tFUNC=sqrt(x*x+y*y+z*z)\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refx: CUSTOM ...\n",
                   "\tARG=v1.b,v1.c,vr.b,vr.c\n",
                   "\tVAR=y1,z1,y2,z2\n",
                   "\tFUNC=y1*z2-y2*z1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refy: CUSTOM ...\n",
                   "\tARG=v1.a,v1.c,vr.a,vr.c\n",
                   "\tVAR=x1,z1,x2,z2\n",
                   "\tFUNC=x2*z1-x1*z2\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refz: CUSTOM ...\n",
                   "\tARG=v1.a,v1.b,vr.a,vr.b\n",
                   "\tVAR=x1,y1,x2,y2\n",
                   "\tFUNC=x1*y2-x2*y1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refs: CUSTOM ...\n",
                   "\tARG=refx,refy,refz\n",
                   "\tVAR=x,y,z\n",
                   "\tFUNC=sqrt(x*x+y*y+z*z)\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "dc: CUSTOM ...\n",
                   "\tARG=crossx,crossy,crossz,refx,refy,refz,crosss,refs\n",
                   "\tVAR=x1,y1,z1,x2,y2,z2,A,B\n",
                   "\tFUNC=(x1*x2+y1*y2+z1*z2)/A/B\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "# BIAS\n",
                   f"udb: RESTRAINT ARG=dcom KAPPA={k_d} AT={dist}\n",
                   f"uab: RESTRAINT ARG=ang KAPPA={k_a} AT={ang}\n",
                   f"ucb: RESTRAINT ARG=dc KAPPA=100.0 AT={circ_id}\n",
                   "\n",
                   f"PRINT ARG=dcom,ang,dc,udb.bias,uab.bias,ucb.bias STRIDE=100 FILE=us_{id}.dat\n",
                    ]
    with open(plumed_f, 'w') as f:
        f.writelines(plumed_cnt)

def write_sub(f_name, ang_dir, ang_id):
    with open(f_name, 'r' ) as f:
        f_cnt = f.readlines()
    line = f_cnt[-3]
    line = line.replace("$ANG_DIR", str(ang_dir/f"{ang_id}"))
    f_cnt[-3] = line
    with open(f_name, 'w' ) as f:
        f.writelines(f_cnt)
    return None

In [18]:
DISTS = np.linspace(0, 2.5, NWINDOWS_DIST)
ANGS = np.linspace(0, np.pi, NWINDOWS_ANG)

for i, ang in enumerate(ANGS):
    for j, dist in enumerate(DISTS):
        ang_dir = RUN_DIR/f"{i}"
        if not ang_dir.exists():
            ang_dir.mkdir()

        sim_dir = RUN_DIR/f"{i}"/f"{j}"
        if sim_dir.exists():
            is_empty = not any(sim_dir.iterdir())
            if not is_empty:
                #raise RuntimeError(f"Directory {i}/{j} already exists and is not empty.")
                print(f"Directory {sim_dir} exists. Skipping...")
                continue
        else:
            sim_dir.mkdir()

        for file in MOLD_DIR.iterdir():
            shutil.copy(file, sim_dir/file.parts[-1])
        plumed_f = sim_dir/"plumed_us.dat"
        write_plumed(f"{i}_{j}", dist, ang, 250.00, 250.00, 0.5, plumed_f)
        sub_f = sim_dir/f"sub_mdrun_plumed.sh"
        write_sub(sub_f, ANG_DIR, i)
    

Directory /project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/runs/1/0 exists. Skipping...
Directory /project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/runs/1/1 exists. Skipping...
Directory /project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/runs/1/2 exists. Skipping...
Directory /project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/runs/1/3 exists. Skipping...
Directory /project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/runs/1/4 exists. Skipping...
Directory /project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/runs/1/5 exists. Skipping...
Directory /project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/runs/1/6 exists. Skipping...
Directory /project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/runs/1/7 exists. Skipping...
Directory /project2/andr

In [12]:
ANGS = ANGS[1:-1]
ANGS = ANGS[::-1]

for i, ang in enumerate(ANGS):
    for j, dist in enumerate(DISTS):
        ang_id = 18 + i
        ang_dir = RUN_DIR/f"{ang_id}"
        if not ang_dir.exists():
            ang_dir.mkdir()

        sim_dir = RUN_DIR/f"{ang_id}"/f"{j}"
        if sim_dir.exists():
            is_empty = not any(sim_dir.iterdir())
            if not is_empty:
                raise RuntimeError(f"Directory {ang_id}/{j} already exists and is not empty.")
        else:
            sim_dir.mkdir()

        for file in MOLD_DIR.iterdir():
            shutil.copy(file, sim_dir/file.parts[-1])
        plumed_f = sim_dir/"plumed_us.dat"
        write_plumed(f"{ang_id}_{j}", dist, ang, 1000.00, 1000.00, -1, plumed_f)
        sub_f = sim_dir/f"sub_mdrun_plumed.sh"
        write_sub(sub_f, ANG_DIR, ang_id)
    

# Submission

Just a little context manager for changing directories

In [19]:
import os
import contextlib

@contextlib.contextmanager
def working_directory(path):
    """Changes working directory and returns to previous on exit."""
    prev_cwd = Path.cwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(prev_cwd)

In [20]:
import subprocess
import multiprocessing

def call_func(ids):
   i, j = ids
   with working_directory(RUN_DIR/str(i)/str(j)):
      print(f"{i}_{j}: ", end="")
      subprocess.call("sbatch -J GHGGF_us --wait sub_mdrun_plumed.sh", shell=True)
   return None

ids = []
for i in range(NWINDOWS_ANG*2-2):
   for j in range(NWINDOWS_DIST):
      if i in [0, 17]:
         ids.append((i, j))

with multiprocessing.Pool(NUM_SUB) as pool:
   pool.map(call_func, ids)

0_0: 0_2: 0_6: 0_4: 

Submitted batch job 31078439
Submitted batch job 31078440
Submitted batch job 31078441
Submitted batch job 31078442
0_1: 0_5: 0_7: 0_3: Submitted batch job 31078462
Submitted batch job 31078463
Submitted batch job 31078464
Submitted batch job 31078465
0_8: 0_10: 17_0: 17_2: Submitted batch job 31078595
Submitted batch job 31078596
Submitted batch job 31078597
Submitted batch job 31078598
17_1: 0_9: 0_11: 17_3: Submitted batch job 31078617
Submitted batch job 31078618
Submitted batch job 31078619
Submitted batch job 31078620
17_4: 17_6: 17_8: Submitted batch job 31078702
Submitted batch job 31078703
Submitted batch job 31078704
17_10: Submitted batch job 31078732
17_11: 17_5: 17_7: 17_9: Submitted batch job 31078753
Submitted batch job 31078754
Submitted batch job 31078755
Submitted batch job 31078756


In [14]:
import os
import asyncio
import telegram

async def notif(message):
    bot = telegram.Bot("")
    async with bot:
        await bot.send_message(chat_id=149407364, text=message)


await notif(f"{os.getcwd()} is done.")